In [ ]:
# 8 STEP: CREATE DATAFRAME OF PUBLICATIONS
# In this Notebook, the dataframe we will work with for our analysis of gender equality
# will be created. For that, the publications and necessary info will be extracted from 'clean-dblp.xml'
# And completed with information on gender from 'names_cat.csv'

In [ ]:
# >>> Import necessary data

# Import the publication data
import datetime
import xml.etree.ElementTree as ET
parser = ET.XMLParser()
    
file = 'data/clean_dblp.xml'

time = datetime.datetime.now()
print("Starting to parse XML file at {} ...".format(time))
tree = ET.parse(file, parser=parser)
time = datetime.datetime.now()
print("Finished parsing XML file at {} ! ".format(time))
root = tree.getroot()
print("Found {} entries! ".format(len(root)))

In [ ]:
# Import categorized 'names_cat.csv'
import pandas

print("Importing categorized names that were improved twice before... ")
names = pandas.read_csv("data/names_cat_i2.csv", usecols=["name", "n_publs", "likely_gender", "score"])

# Setting index & accessing cells: https://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
names = names.set_index("name", drop = False)
print("Names imported. They look like this: {}".format(names[:10]))

In [ ]:
# >>> Set the min. reliable score (found out in last step)
score_min = 5
change_fact = score_min - 1

In [ ]:
# >>> Create a dataset of adjusted scores 
# These will be used to try and find out which score is the most reliable
def adjust(score):
    if(score >= score_min):
        return score - change_fact # any score that is reliable is 1 or bigger
    else:
        return 0 # unreliable scores are all 0

adjusted_scores = []
for score in names['score']:
    adjusted_scores.append(adjust(score))

In [ ]:
# Visualize adjusted scores
import matplotlib.pyplot as plt
names['score'].hist(bins=names['score'].max())
plt.hist(adjusted_scores, bins=max(adjusted_scores), alpha=0.5)

In [ ]:
# >>> Calc outliers 
# https://www.wikihow.com/Calculate-Outliers
'''n = names_scores_new
med = np.median(n)
higher =  np.percentile(n, 75)
lower =  np.percentile(n, 25)
interquart_range = higher - lower
inner_fence_higher = higher + interquart_range * 1.5
outer_fence_higher = higher + interquart_range * 3
print("For the adjusted scores, the inner fence is {}, the outer fence is {}".format(inner_fence_higher, outer_fence_higher))

inner_fence_lower = lower - interquart_range * 1.5
outer_fence_lower = lower -  interquart_range * 3
print("For the adjusted scores, the inner fence is {}, the outer fence is {}".format(inner_fence_lower, outer_fence_lower))'''

In [ ]:
import numpy as np

# >>> Get all publications and fill with info

# Get all gender related info
# The gender inequality index r tells us how much equalizing would be needed so there is an equal amount of male and 
# female authors. For example, if there are 4 authors in total, and 3 are male, 1 would need to be female in order 
# for the ratio between man and women to be equal. 
# It is calculated like so: 50% (the half) - 25% (percentage of women) = 25% (inequality index)

higher =  np.percentile(adjusted_scores, 75)
score_max = higher + change_fact # calculate back TODO: Is this the right value to chose? Currently 7
def get_gender_picture(n):
    "This function returns the amount of male, female and unknown authors as well as the gender inequality ratio (in %)"
    m_min = 0.0 #male
    m_max = 0.0
    f_min = 0.0 #female
    f_max = 0.0
    u = 0.0 #unknown
    r_min = 0.0 #inequality index
    r_max = 0.0
    for name in n:
        score = names.at[name, "score"]
        gender = names.at[name, "likely_gender"]
        if(score >= score_min):
            if(gender == "male"):
                m_max += 1.0
            elif(gender == "female"):
                f_max += 1.0
        if(score > score_max):
            if(gender == "male"):
                m_min += 1.0
            elif(gender == "female"):
                f_min += 1.0
    sm_total = len(n)
    
    sm_min = m_min + f_min #sum of male and female authors
    sm_max = m_max + f_max
    
    u_min = sm_total - sm_max
    u_max = sm_total - sm_min

    # Inequality Index
    
    half_rel = 50
    if(sm_max > 0):
        f_rel_max = (f_min / sm_max) * 100 #relative amount of female authors
        r_max = half_rel - f_rel_max #inequality index
    if(sm_min > 0):
        f_rel_min = (f_min / sm_min) * 100 #relative amount of female authors
        r_min = half_rel - f_rel_min #inequality index
    
    # Contributions
    if(sm_total > 0):
        worth_contrib = 1.0 / sm_total
    else:
        worth_contrib = 0
    
    n_contribs_f_min = f_min * worth_contrib
    n_contribs_m_min = m_min * worth_contrib
    n_contribs_u_max = u_max * worth_contrib
    
    n_contribs_f_max = f_max * worth_contrib
    n_contribs_m_max = m_max * worth_contrib
    n_contribs_u_min = u_min * worth_contrib
   
    return {
        'males': m_max,
        'females': f_max,
        'unknowns': u_min,
        'ratio': r_max,
        
        'males_min': m_min,
        'females_min': f_min,
        'unknowns_max': u_max,
        'ratio_min': r_min,
        
        'contribs_f': n_contribs_f_max,
        'contribs_m': n_contribs_m_max,
        'contribs_u': n_contribs_u_min,
        
        'contribs_f_min': n_contribs_f_min,
        'contribs_m_min': n_contribs_m_min,
        'contribs_u_min': n_contribs_u_max
    }

In [ ]:
# Get all publications
publications_complete = {}

print("Starting to extract publications...")

for child in root:
    # Extract: key, category of publication, 
    key = child.get('key')
    pub_cat = child.tag
    
    # Optionally extract: title, year of publication, publtype, pages
    titles = child.findall("title")
    if(len(titles) > 0): title = titles[0].text
    
    years = child.findall("year")
    if(len(years) > 0): year = years[0].text
    
    pub_type = child.get('publtype')
    
    # Get authors
    authors = child.findall("author")
    
    current_names = []
    
    for author in authors:
        current_names.append(author.text)
    
    # Get gender info
    n_authors = len(current_names)
    
    gender_pic = get_gender_picture(current_names)
    n_males = gender_pic['males']
    n_females = gender_pic['females']
    n_unknown = gender_pic['unknowns']
    ratio = gender_pic['ratio']
    
    n_males_min = gender_pic['males_min']
    n_females_min = gender_pic['females_min']
    n_unknown_max = gender_pic['unknowns_max']
    ratio_min = gender_pic['ratio_min']
    
    n_contribs_f_min = gender_pic['contribs_f_min']
    n_contribs_m_min = gender_pic['contribs_m_min']
    n_contribs_u_max = gender_pic['contribs_u']
    
    n_contribs_f_max = gender_pic['contribs_f']
    n_contribs_m_max = gender_pic['contribs_m']
    n_contribs_u_min = gender_pic['contribs_u_min']
    
    # Add publication entry
    publications_complete[key] = {
        'key': key,
        'pub_cat': pub_cat,
        'pub_type': pub_type,
        'title': title,
        'year': year,
        
        'authors': current_names,
        'n_authors': n_authors,
        
        'n_males': n_males,
        'n_females': n_females,
        'n_unknown': n_unknown,
        'ratio': ratio,
        
        'n_males_min': n_males_min,
        'n_females_min': n_females_min,
        'n_unknown_max': n_unknown_max,
        'ratio_min': ratio_min,
        
        'n_contribs_females': n_contribs_f_max,
        'n_contribs_males': n_contribs_m_max,
        'n_contribs_unknown': n_contribs_u_min,
        
        'n_contribs_females_min': n_contribs_f_min,
        'n_contribs_males_min': n_contribs_m_min,
        'n_contribs_unknown_max': n_contribs_u_max
    }

print("Finished extracting publications and calculating all needed information! Found {} entries.".format(len(publications_complete)))

In [ ]:
# >>> Convert results into dataframe
# Dict to dataframe: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html
print("Converting dictionary of publications to a dataframe...")
publications = pandas.DataFrame.from_dict(publications_complete, orient='index', dtype=None)
print("Dataframe completed! Here are some entries: {}".format(publications[:7]))

In [ ]:
# >>> Save results
# Save Dataframe to CSV: https://riptutorial.com/pandas/example/19502/create-random-dataframe-and-write-to--csv
print("Saving dataframe as CSV...")
publications.to_csv("data/publications_cor2.csv")
print("Publications saved as CSV!")